In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import time


UNATTENDED_COST = 10000
VEHICLES_COST = 1000
WAIT_DELAY_COST = 10


# Dataset class remains the same
class VRPTWDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file)
        self.data = df
        self.coordinates = self.data[['X', 'Y']].values
        self.time_windows = self.data[['READY_TIME', 'DUE_TIME']].values
        self.service_times = self.data['SERVICE_TIME'].values
        self.num_customers = len(self.data) - 1  # Exclude the depot

    def __len__(self):
        return self.num_customers

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        coord = self.coordinates[idx + 1]  # +1 because the first row is depot
        time_window = self.time_windows[idx + 1]
        service_time = self.service_times[idx + 1]
        sample = {'coord': coord, 'time_window': time_window, 'service_time': service_time}
        return sample

# Plotting the routes
def plot_routes(routes, dataset):
    plt.figure(figsize=(10, 10))
    depot = dataset.coordinates[0]
    plt.scatter(depot[0], depot[1], c='red', s=200, label='Depot')

    # Plot and label each customer's location
    for i, route in enumerate(routes):
        route_coords = [depot] + [dataset[customer]['coord'] for customer in route] + [depot]
        route_coords = np.array(route_coords)
        plt.plot(route_coords[:, 0], route_coords[:, 1], marker='o', label=f'Vehicle {i+1}')

        # Label each customer with its ID
        for j, customer_idx in enumerate(route):
            plt.text(route_coords[j+1, 0], route_coords[j+1, 1], f'{customer_idx+1}', fontsize=12, ha='right')

    # Label the depot as well
    plt.text(depot[0], depot[1], 'Depot', fontsize=12, ha='right')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend(fontsize=8)  # Adjust the legend fontsize here
    plt.title('Vehicle Routes with Customer IDs')
    plt.show()


# Improved cost function with penalties
def calculate_cost(route, dataset, remaining_customers):
    depot = dataset.coordinates[0]
    current_time = 0
    current_location = depot
    cost = 0
    num_vehicles_used = 1 if route else 0  # Penalización por cada vehículo usado

    for customer_idx in route:
        customer = dataset[customer_idx]
        travel_time = np.linalg.norm(current_location - customer['coord'])
        arrival_time = current_time + travel_time

        # Penalización por llegar antes o después de la ventana de tiempo
        early_arrival_penalty = max(0, customer['time_window'][0] - arrival_time) * WAIT_DELAY_COST
        late_arrival_penalty = max(0, arrival_time - customer['time_window'][1]) * WAIT_DELAY_COST
        time_window_penalty = early_arrival_penalty + late_arrival_penalty

        start_service_time = max(arrival_time, customer['time_window'][0])
        current_time = start_service_time + customer['service_time']
        current_location = customer['coord']
        cost += travel_time + customer['service_time'] + time_window_penalty

    return_to_depot_time = np.linalg.norm(current_location - depot)
    cost += return_to_depot_time
    return cost, num_vehicles_used

# Modified greedy heuristic to keep track of unserved customers and vehicle count
def greedy_heuristic_solution(dataset, num_vehicles):
    depot = dataset.coordinates[0]
    customers = list(range(len(dataset)))
    routes = [[] for _ in range(num_vehicles)]
    remaining_customers = set(customers)

    for vehicle in range(num_vehicles):
        current_time = 0
        current_location = depot

        while remaining_customers:
            feasible_customers = []
            for customer_idx in remaining_customers:
                customer = dataset[customer_idx]
                travel_time = np.linalg.norm(current_location - customer['coord'])
                arrival_time = current_time + travel_time

                if arrival_time <= customer['time_window'][1]:
                    feasible_customers.append((customer_idx, arrival_time, travel_time))

            if not feasible_customers:
                break

            # Select the nearest feasible customer
            feasible_customers.sort(key=lambda x: x[2])  # Sort by travel time
            selected_customer = feasible_customers[0][0]

            routes[vehicle].append(selected_customer)
            remaining_customers.remove(selected_customer)

            # Update the vehicle's current time and location
            travel_time = feasible_customers[0][2]
            current_time = max(current_time + travel_time, dataset[selected_customer]['time_window'][0]) + dataset[selected_customer]['service_time']
            current_location = dataset[selected_customer]['coord']

    unserved_customers_penalty = len(remaining_customers) * UNATTENDED_COST
    return routes, unserved_customers_penalty

# Modified cost function to return detailed information
def calculate_route_details(route, dataset, vehicle):
    depot = dataset.coordinates[0]
    current_time = 0
    current_location = depot
    accumulated_cost = 0
    total_distance = 0
    total_wait_delay = 0
    route_details = []
    from_customer = 'Depot'

    for i, customer_idx in enumerate(route):
        customer = dataset[customer_idx]
        travel_time = np.linalg.norm(current_location - customer['coord'])
        total_distance += travel_time
        arrival_time = current_time + travel_time

        # Calculate waiting time, delay, and penalties
        waiting_time = max(0, customer['time_window'][0] - arrival_time)
        delay = max(0, arrival_time - customer['time_window'][1])
        start_service_time = max(arrival_time, customer['time_window'][0])
        end_service_time = start_service_time + customer['service_time']

        # Calculate penalties and add them to the cost
        time_window_penalty = (waiting_time + delay) * 10
        total_wait_delay += time_window_penalty
        segment_cost = travel_time  + time_window_penalty
        accumulated_cost += segment_cost

        # Add route detail row
        route_details.append({
            'Vehicle': vehicle,
            'From Customer': 'Depot: (0,0)' if i == 0 else f"{from_customer}: ({dataset[from_customer]['coord'][0]}, {dataset[from_customer]['coord'][1]})",
            'To Customer': f"{customer_idx}: ({customer['coord'][0]}, {customer['coord'][1]})",
            'Travel Starts At': current_time,
            'Arrival Time': arrival_time,
            'Distance': travel_time,
            'Ready Time': customer['time_window'][0],
            'Due Time': customer['time_window'][1],
            'Service Time': customer['service_time'],
            'Waiting Time': waiting_time,
            'Delay': delay,
            'Ends Delivery At': end_service_time,
            'Accumulated Cost': accumulated_cost,
            'Accumulated Distance': round(total_distance, 2) # Añadir distancia acumulada
        })

        # Update current time and location
        current_time = end_service_time
        current_location = customer['coord']
        from_customer = customer_idx

    if route and customer_idx:
      # Add the return to depot
      return_to_depot_time = np.linalg.norm(current_location - depot)
      accumulated_cost += return_to_depot_time
      total_distance += return_to_depot_time

      route_details.append({
          'Vehicle': vehicle,
          'From Customer': f"{customer_idx}: ({customer['coord'][0]}, {customer['coord'][1]})",
          'To Customer': 'Depot: (0,0)',
          'Travel Starts At': current_time,
          'Arrival Time': current_time + return_to_depot_time,
          'Distance': return_to_depot_time,
          'Ready Time': None,
          'Due Time': None,
          'Service Time': None,
          'Waiting Time': 0,
          'Delay': 0,
          'Ends Delivery At': current_time + return_to_depot_time,
          'Accumulated Cost': accumulated_cost,
          'Accumulated Distance': round(total_distance, 2) # Añadir distancia acumulada
      })

    return route_details, total_distance, total_wait_delay, accumulated_cost

# Main function to solve the VRPTW problem with detailed tables
def solve_vrptw(csv_file, num_vehicles=5, execution=0):
    dataset = VRPTWDataset(csv_file)
    num_customers = len(dataset)

    # Use a greedy heuristic to generate routes
    best_routes, unserved_customers_penalty = greedy_heuristic_solution(dataset, num_vehicles)

    total_cost = unserved_customers_penalty
    num_vehicles_used = 0
    df_list = []
    total_distance = 0
    total_wait_delay = 0

    for vehicle_idx, route in enumerate(best_routes):
        route_details, route_distance, wait_delay, route_cost = calculate_route_details(route, dataset, vehicle_idx+1)
        if route_details:
          total_cost += route_cost  # Add the cost of the route to the total cost
          num_vehicles_used += 1 if route else 0
          total_distance += route_distance
          total_wait_delay += wait_delay

          df_list.append((vehicle_idx+1, pd.DataFrame(route_details)))

    # Add penalty for the number of vehicles used
    total_cost += num_vehicles_used * VEHICLES_COST

    print("Total Cost:", total_cost)
    print("Total Distance:", total_distance)
    print("Total Wait/Delay Cost:", total_wait_delay)
    print(f"Unserved Customers Penalty: {unserved_customers_penalty}")
    print(f"Vehicles Cost: {num_vehicles_used * VEHICLES_COST}")
    elapsed_time = time.time() - problem_start_time
    print(f"Execution time for problem {i}: {elapsed_time:.2f} seconds")

    all_routes_df = pd.DataFrame()
    #plot_routes(best_routes, dataset)
    for (vehicle_idx, df) in df_list:
      # print(f"Route Details for Vehicle {vehicle_idx + 1}")
      # print(df)
      all_routes_df = pd.concat([all_routes_df, df], ignore_index=True)

    all_routes_df.to_csv(f"HEU_{problem}_execution_{execution}.csv", index=False)

    return total_cost, total_distance, total_wait_delay, num_vehicles_used * VEHICLES_COST, unserved_customers_penalty, elapsed_time


NUM_EXECUTIONS = 30
PROBLEM_LIST = [
    'vrptw_10_customers_problem_1.csv',
    'vrptw_10_customers_problem_3.csv',
    'vrptw_10_customers_problem_4.csv',
    'vrptw_10_customers_problem_5.csv',
    'vrptw_10_customers_problem_8.csv',
    'vrptw_10_customers_problem_9.csv',
    'vrptw_20_customers_problem_1.csv',
    'vrptw_20_customers_problem_3.csv',
    'vrptw_20_customers_problem_5.csv',
    'vrptw_20_customers_problem_8.csv',
    'vrptw_20_customers_problem_9.csv',
    'vrptw_20_customers_problem_10.csv',
]

if __name__ == '__main__':
    # Ejecución de los problemas
    for problem in PROBLEM_LIST:
      results = []
      N_VEHICLES = 5 if "_10_" in problem else 10
      print(f"Running problem {problem}")
      for i in range(1, NUM_EXECUTIONS + 1):
        global problem_start_time
        problem_start_time = time.time()
        # Ejecución del modelo
        cost, distance_cost, wait_delay_cost, vehicles_cost, unattended_cost, elapsed_time = solve_vrptw(f'./{problem}', num_vehicles=N_VEHICLES, execution=i)
        print(f"Execution {i} - time: {elapsed_time:.2f} seconds")
        print("\n")
        results.append((i, problem, cost, distance_cost, wait_delay_cost, vehicles_cost, unattended_cost, elapsed_time, N_VEHICLES))

      COLUMNS = ['Execution', 'Problem', 'Cost', 'Distamce', 'Wait_Delay_Cost', 'Vehicles_Cost', 'Unattended_Cost', 'Time', 'N_Vehicles']
      df = pd.DataFrame(results, columns = COLUMNS)
      df.to_csv(f'./HEU_results_{problem}', index=False)
